In [ ]:

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install scipy==1.1.0
!pip install pandas==0.23.4
!pip install numpy==1.15.2
!pip install seaborn==0.9.0
!pip install matplotlib==3.0.0
!pip install imbalanced_learn==0.4.3
!pip install xgboost==0.90
!pip install lightgbm==2.2.3
!pip install imblearn==0.0
!pip install scikit_learn==0.21.3


In [ ]:
!pip install numpy --upgrade
!pip install --upgrade pandas
!pip install lightgbm --upgrade
!pip install -U scikit-learn

In [ ]:
!pip install six
!pip install django_taggit==1.2.0


In [ ]:
import argparse
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import precision_recall_curve, confusion_matrix, classification_report, roc_curve, auc, f1_score, recall_score, accuracy_score, mean_squared_error
from sklearn.model_selection import cross_val_score, RandomizedSearchCV, train_test_split
from sklearn.preprocessing import MinMaxScaler
import statistics
import six
from imblearn.over_sampling import SMOTE
import xgboost as xgb
from scipy.stats import uniform, randint
from scipy import interp
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import ElasticNet
from sklearn.model_selection import RepeatedKFold
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

In [ ]:
!pip install feature_selector

In [ ]:
pip install --upgrade pip

In [ ]:
from feature_selector import FeatureSelector

In [ ]:
!pip install --upgrade pandas

In [ ]:
!pip install numpy --upgrade

In [ ]:
!pip install lightgbm --upgrade


In [ ]:
#################
### Arguments ###
#################

parser = argparse.ArgumentParser(description='EGFR and KRAS Lung Cancer Mutation Status - Train')
parser.add_argument('--gene', type=str,
                    default='EGFR',
                    help='pick gene - EGFR / KRAS (default: EGFR)')
parser.add_argument('--path-EGFR', type=str,
                    default=r"/content/drive/MyDrive/Colab_Notebooks/lucasradsemegfrkras-master/Radiomics/data/EGFR/FEATURES.xlsx",
                    help='data path for EGFR (default:/content/drive/MyDrive/Colab_Notebooks/lucasradsemegfrkras-master/Radiomics/data/EGFR/FEATURES.xlsx)' )
parser.add_argument('--path-KRAS', type=str,
                    default=r"/content/drive/MyDrive/Colab_Notebooks/lucasradsemegfrkras-master/Radiomics/data/EGFR/FEATURES.xlsx)'",
                    help='data path for KRAS (default: /content/drive/MyDrive/Colab_Notebooks/lucasradsemegfrkras-master/Radiomics/data/EGFR/FEATURES.xlsx)')
parser.add_argument('--n-tests', type=int,
                    default=50,
                    help='number of tests (default: 100)')
args = parser.parse_args("")

In [ ]:
#################
### Functions ###
#################

def plot_confusion_matrix(y_true, y_pred, classes,
                          normalize=False,
                          title=None,
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if not title:
        if normalize:
            title = 'Normalized confusion matrix'
        else:
            title = 'Confusion Matrix'

    # Compute confusion matrix
    cm = confusion_matrix(y_true, y_pred)

    # Only use the labels that appear in the data
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion Matrix')

    print(cm)

    fig, ax = plt.subplots()
    im = ax.imshow(cm, interpolation='nearest', cmap=cmap)
    ax.figure.colorbar(im, ax=ax)
    # We want to show all ticks...
    ax.set(xticks=np.arange(cm.shape[1]),
           yticks=np.arange(cm.shape[0]),
           # ... and label them with the respective list entries
           xticklabels=classes, yticklabels=classes,
           title=title,
           ylabel='True label',
           xlabel='Predicted label')

    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_yticklabels(), rotation=90, va="center")

    # Loop over data dimensions and create text annotations.
    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(j, i, format(cm[i, j], fmt),
                    ha="center", va="center",
                    color="white" if cm[i, j] > thresh else "black")
    fig.tight_layout()
    plt.savefig('confusionmatrix.jpg')
    return ax

In [ ]:
############
### Data ###
############

if args.gene=='EGFR':
    # Path of the features regarding EGFR mutation status
    features = pd.read_excel(args.path_EGFR)

    labels = np.array(features['EGFR mutation status'])
    ids=np.array(features['Case ID'])

    # Remove labels from the features
    features= features.drop(['EGFR mutation status', 'Case ID'], axis = 1)
elif args.gene=='KRAS':
    # Path of the features regarding KRAS mutation status
    features = pd.read_excel(args.path_KRAS)

    # Labels are the values we want to predict
    labels = np.array(features['KRAS mutation status'])
    ids=np.array(features['Case ID'])

    # Remove labels from the features
    features= features.drop(['KRAS mutation status', 'Case ID'], axis = 1)
    # tira as duas colunas q não são features
else:
    print("Error: Invalid gene!")

#print(features)

# One-hot encode categorical features
features = pd.get_dummies(features, prefix=['Gender', 'Smoking status'])
# Binariza as features de género e smoking status para 4: M, F, S, NS

print('The shape of our features is:', features.shape)

# Saving feature names for later use
feature_list = list(features.columns)
# guarda o nome de todas as features numa lista

# Convert to numpy array
features = np.array(features)
print(features)

In [ ]:
################
### Training ###
################
FPRXBTotal=[]
AucXBTotal=[]
F1XBTotal=[]
SensitivityXBTotal=[]
PrecisionXBTotal=[]
SpecificitynXBTotal=[]
tprs = []
aucs = []
mean_fpr = np.linspace(0, 1, 500) # cria um np array com 500 nºs entre 0 e 1

# Make n_tests splits, training xgboost with Cross Validation hyperparameter tuning for each one
for i in range(0, args.n_tests):

    print('######## Test number {} ########\n'.format(i+1))

    # Split the data into training set and test set (80/20% division)
    X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, shuffle=True, stratify=labels)
    # função do scikit_learn.model_selection

    # Normalization - only radiomic features are normlaized, clinical are not considered
    cols_to_norm = np.arange(0,features.shape[1]-5)
    scaler = MinMaxScaler()
    X_train[:,cols_to_norm] = scaler.fit_transform(X_train[:,cols_to_norm])
    X_test[:,cols_to_norm] = scaler.transform(X_test[:,cols_to_norm])

    # Create pandas data frame
    X_train=pd.DataFrame(X_train, columns=feature_list)
    y_train=pd.DataFrame(y_train)
    X_test=pd.DataFrame(X_test, columns=feature_list)
    y_test=pd.DataFrame(y_test)

    #X_train = np.array(X_train)
    #Q, R= np.linalg.qr(X_train)
    #min_tol = 1e-9
    #indep = np.where(np.abs(R.diagonal()) >  min_tol)[0]
    #print(indep)
    #print(X_train[:, indep])

    #X_train = np.array(X_train)
   # X_test=np.array(X_test)
   # Q, R= np.linalg.qr(X_train)
    #min_tol = 1e-9
   # indep = np.where(np.abs(R.diagonal()) >  min_tol)[0]
   # X_train= X_train[:, indep]
    #Q1, R1= np.linalg.qr(X_test)
    #indep = np.where(np.abs(R1.diagonal()) >  min_tol)[0]
   # X_test= X_test[:, indep]

    #########PCA######################
    pca_cancer = PCA(n_components=0.75)
    X_train = pca_cancer.fit_transform(X_train)
    X_test = pca_cancer.transform(X_test)

    # Create Feature Selector instance
    #fs = FeatureSelector(data = X_train, labels = y_train)

    #PAIRWISE CORRELATION FILTER!!!!!!!!!!!!!!!
    # Remove higlhy correlated features
    #fs.identify_collinear(correlation_threshold=0.95)
    #correlated_features = fs.ops['collinear']
    #train_no_collinear = fs.remove(methods = ['collinear'])
    #test_no_collinear = X_test.drop(correlated_features, axis=1)
    #collinear=len(correlated_features)
    # Remove zero and low importance features
    # NOTE: Change the hyper-parameters values of the GBM in the feature_selector.py file
    # EGFR---> n_estimators:1000, learning rate: 0.02
    # KRAS---> n_estimators:400, learning rate: 0.1

    # Number of features after removing collinear features
    #print('Number of features after removing collinear features:', train_no_collinear.shape)

    #REMOVE ZERO IMPORTANCE FEATURES!!!!!!!!!!!!!!!
    # Create new Feature Selector instance
    #fs = FeatureSelector(data = train_no_collinear, labels = y_train)

    # Identify zero importance features
   # fs.identify_zero_importance(task = 'classification',
         #                   eval_metric = 'auc',
        # #                   n_iterations = 10,
           #                  early_stopping = True)

    #Identify low importance features
    #fs.identify_low_importance(cumulative_importance = 0.95)
    #zero_importance_features = fs.ops['zero_importance']
    #zero=len(zero_importance_features)


    #removed_features.append(collinear+zero)

    #Remove features from training set
    #final_training_set = fs.remove(methods = ['zero_importance'])

    #Removing features from test set
    #final_test_set = test_no_collinear.drop(zero_importance_features, axis=1)

    #Number of features after removing zero and low importance features
    #print('Number of features after removing zero and low importance features:', len(final_training_set.columns))

    # Saving feature names for later use
    #feature_list_final = list(final_training_set.columns)

    # Convert to numpy array
    X_train = np.array(X_train)
    X_test = np.array(X_test)

    print('Training Features Shape:', X_train.shape)
    print('Testing Features Shape:', X_test.shape)

    # Data augmentation using SMOTE
    # NOTE: In both EGFR and KRAS cases none of the clinical features
    # is preserved after feature selection. That's why SMOTE is used
    # instead of SMOTE-NC
    sm = SMOTE(random_state=42)
    X_train_res, y_train_res = sm.fit_resample(X_train, y_train)

    print('Number of samples after SMOTE:', X_train_res.shape)

    X_train = np.array(X_train)
    X_test=np.array(X_test)

    # define SVN


    #svclassifier = SVC(kernel='linear', probability=True)
    #param_grid = {'C': [0.1,1, 10, 100], 'gamma': [1,0.1,0.01,0.001]}
    #search = GridSearchCV(svclassifier,param_grid,refit=True,verbose=2)
    svclassifier = SVC(kernel='poly', probability=True)

    param_grid = {'C': [0.1,1, 10, 100], 'gamma': [1,0.1,0.01,0.001], 'degree':[2,3] }

    # Intialize and fit randomized search
    search = GridSearchCV(svclassifier,param_grid,refit=True,verbose=2)
    #search.fit(X_train_res, y_train_res)
    #clf = RandomForestClassifier(random_state=42)

    #params = {
    #"max_depth": [5, 8, 15, 25, 30],
    #"n_estimators": [100, 300, 500, 800, 1200],
    #"max_features": ["auto", "sqrt"],
    #"criterion": ["gini", "entropy"],
    #"bootstrap": [True, False],
    #"min_samples_leaf": [1, 2, 5, 10],
    #"min_samples_split": [2, 5, 10, 15, 100],
    #}
    #search = RandomizedSearchCV(clf, param_distributions=params, n_iter=15, scoring='roc_auc', cv=5, verbose=3)
    #search.fit(X_train_res, y_train_res)

    #dual=[True,False]
    #max_iter=[100,110,120,130,140]
    #C = [0.01, 0.1, 1.0,1.5,2.0,2.5, 10, 100]
    #param_grid = dict(dual=dual,max_iter=max_iter,C=C)

    #lr = LogisticRegression(penalty='l2')
    #search = GridSearchCV(estimator=lr, param_grid=param_grid, scoring = 'roc_auc', cv=5)
        # define ElasticNet
    #ENclassifier = ElasticNet()

    #parametersGrid = {"max_iter": [1, 5, 10],
        #              "alpha": [0.0001, 0.001, 0.01, 0.1, 1, 10, 100],
        #              "l1_ratio": np.arange(0.0, 1.0, 0.1)}

    #cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=42)

    # Intialize and fit randomized search
    #search = GridSearchCV(ENclassifier,parametersGrid,cv=cv, scoring='roc_auc')
    #search.fit(X_train_res, y_train_res)

    # Print best hyperparameters according to CV
    #print("Best parameters set found on development set:")
    #print()
    #print(search.best_params_)
    #print()
    #print(search.best_estimator_)

     #define xgboost
    #xgb_model = xgb.XGBClassifier(nthread=1, learning_rate=0.01, n_estimators=600, objective="binary:logistic", random_state=42)

    #params = {
       #"classification__colsample_bytree": uniform(0.3, 0.7),
      ## "classification__gamma": uniform(0.5, 2),
       #"classification__subsample": uniform(0.6, 0.4),
      # "classification__learning_rate": uniform(0.03, 0.3),
      # "classification__max_depth": randint(2, 6),
      # "classification__n_estimators": randint(100, 1000),
      # 'classification__min_child_weight': randint(1, 5)
  # }

     #Intialize and fit randomized search
    #search = RandomizedSearchCV(xgb_model, param_distributions=params, n_iter=15, scoring='roc_auc', n_jobs=1, cv=5, verbose=3)
    search.fit(X_train_res, y_train_res)



    # Test XGBOOST
    y_pred=search.predict_proba(X_test)[:,1]


    #ROC curve
    fpr, tpr, thresholds = roc_curve(y_test, y_pred)
    print('AUC:', auc(fpr, tpr))
    AucXBTotal.append(auc(fpr, tpr))

In [ ]:
print("Mean AUC for XB", sum(AucXBTotal)/len(AucXBTotal), "+\-", statistics.stdev(AucXBTotal))


In [ ]:
F1XBTotal

In [ ]:
a=[0.3636363636363636,
 0.5,
 0.3076923076923077,
 0.625,
 0.3636363636363636,
 0.20000000000000004,
 0.6153846153846154,
 0.14285714285714285,
 0.3076923076923077,
 0.4444444444444445,
 0.6,
 0.6,
 0.4444444444444444,
 0.5333333333333333,
 0.28571428571428575,
 0.3636363636363636,
 0.3636363636363636,
 0.3076923076923077,
 0.5,
 0.3636363636363636,
 0.1818181818181818,
 0.5714285714285714,
 0.28571428571428575,
 0.5454545454545454,
 0.6,
 0.42857142857142855,
 0.5454545454545454,

 0.5,
 0.3333333333333333,
 0.4444444444444445,
 0.25,
 0.5,
 0.4615384615384615,
 0.3636363636363636,

 0.16666666666666666,
 0.26666666666666666,
 0.6,

 0.4615384615384615,
 0.5714285714285714,
 0.20000000000000004,
 0.25,
 0.5714285714285714,

 0.5454545454545454,
 0.3076923076923077,
 0.22222222222222224,
 0.3333333333333333]

In [ ]:
sum(a)/len(a)

In [ ]:
statistics.stdev(a)